<a href="https://colab.research.google.com/github/Shujaat123/ACP_DeepSRC/blob/main/ACP_DeepSRC_Kfold_Stratified_SKv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys, os, re, gc
import numpy as np
import pandas as pd
from random import sample

## Models
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras import metrics
from keras import optimizers
from keras.utils import to_categorical # for keras > 2.0

## Perfmetrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

In [ ]:
def SRC_Pred(y_latent_pred):
    y_pred=[]
    for i in range(y_latent_pred.shape[1]):

        if (LA.norm(y_latent_pred[i][y_train==1])>= LA.norm(y_latent_pred[i][y_train==0])):
            y_pred.append(1)
        else:
            y_pred.append(0)

    return np.array(y_pred)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
## Designing an Auto-Encoder-Classifier model
def Deep_SRC(input_shape=840, LV=600):
    # Encoder Network
    enc_input = Input(shape=(input_shape,), name='enc_input')
    enc_l1 = Dense(100, activation='relu', name='encoder_layer1')(enc_input)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = Dropout(rate = 0.3)(enc_l1)

    enc_l2 = Dense(100, activation='relu', name='encoder_layer2')(enc_l1)
    enc_l2 = BatchNormalization()(enc_l2)
    enc_l2 = Dropout(rate = 0.3)(enc_l2)

    enc_l3 = Dense(100, activation='relu', name='encoder_layer3')(enc_l2)
    enc_l3 = BatchNormalization()(enc_l3)
    enc_l3 = Dropout(rate = 0.3)(enc_l3)

    enc_l4 = Dense(100, activation='relu', name='encoder_layer4')(enc_l3)
    enc_l4 = BatchNormalization()(enc_l4)
    enc_l4 = Dropout(rate = 0.3)(enc_l4)

    encoder_output = Dense(LV, activation='softmax', name='encoder_output')(enc_l4)

    # # Classifier Network
    #class_output = Dense(2, activation='softmax', name='class_layer1')(encoder_output)

    # Decoder Network
    dec_l1 = Dense(100, activation='relu', name='decoder_layer1')(encoder_output)
    dec_l1 = BatchNormalization()(dec_l1)
    dec_l1 = Dropout(rate = 0.3)(dec_l1)

    dec_l2 = Dense(100, activation='relu', name='decoder_layer2')(dec_l1)
    dec_l2 = BatchNormalization()(dec_l2)
    dec_l2 = Dropout(rate = 0.3)(dec_l2)

    dec_l3 = Dense(100, activation='relu', name='decoder_layer3')(dec_l2)
    dec_l3 = BatchNormalization()(dec_l3)
    dec_l3 = Dropout(rate = 0.3)(dec_l3)

    dec_l4 = Dense(100, activation='relu', name='decoder_layer4')(dec_l3)
    dec_l4 = BatchNormalization()(dec_l4)
    dec_l4 = Dropout(rate = 0.3)(dec_l4)

    decoder_output = Dense(input_shape, activation='sigmoid', name='decoder_output')(dec_l4)

    model = Model(inputs=[enc_input], outputs=[encoder_output, decoder_output])

    # Compiling model
    model.compile(optimizer='adam',#optimizer='rmsprop',
                  loss={'encoder_output': 'categorical_crossentropy', 'decoder_output': 'mean_squared_error'},
                  loss_weights={'encoder_output': 0.001, 'decoder_output': 0.999},
                  metrics=[metrics.categorical_accuracy])
    return model

In [ ]:
## Designing an Auto-Encoder-Classifier model
def Deep_VAE_SRC(input_shape =840, LV=600):
    # Encoder Network
    enc_input = Input(shape=(input_shape,), name='enc_input')
    enc_l1 = Dense(100, activation='relu', name='encoder_layer1')(enc_input)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = Dropout(rate = 0.3)(enc_l1)

    enc_l2 = Dense(100, activation='relu', name='encoder_layer2')(enc_l1)
    enc_l2 = BatchNormalization()(enc_l2)
    enc_l2 = Dropout(rate = 0.3)(enc_l2)

    enc_l3 = Dense(100, activation='relu', name='encoder_layer3')(enc_l2)
    enc_l3 = BatchNormalization()(enc_l3)
    enc_l3 = Dropout(rate = 0.3)(enc_l3)

    # enc_l4 = Dense(100, activation='relu', name='encoder_layer4')(enc_l3)
    # enc_l4 = BatchNormalization()(enc_l4)
    # enc_l4 = Dropout(rate = 0.3)(enc_l4)

    #encoder_output = Dense(LV, activation='softmax', name='encoder_output')(enc_l4)
    z_mean = Dense(LV, name="z_mean")(enc_l3)
    z_log_var = Dense(LV, name="z_log_var")(enc_l3)
    encoder_output = Sampling()([z_mean, z_log_var])
    encoder_output = Dense(LV, activation='relu', name='encoder_output')(encoder_output)

    # # Classifier Network
    #class_output = Dense(2, activation='softmax', name='class_layer1')(encoder_output)

    # Decoder Network
    dec_l1 = Dense(100, activation='relu', name='decoder_layer1')(encoder_output)
    dec_l1 = BatchNormalization()(dec_l1)
    dec_l1 = Dropout(rate = 0.3)(dec_l1)

    dec_l2 = Dense(100, activation='relu', name='decoder_layer2')(dec_l1)
    dec_l2 = BatchNormalization()(dec_l2)
    dec_l2 = Dropout(rate = 0.3)(dec_l2)

    dec_l3 = Dense(100, activation='relu', name='decoder_layer3')(dec_l2)
    dec_l3 = BatchNormalization()(dec_l3)
    dec_l3 = Dropout(rate = 0.3)(dec_l3)

    dec_l4 = Dense(100, activation='relu', name='decoder_layer4')(dec_l3)
    dec_l4 = BatchNormalization()(dec_l4)
    dec_l4 = Dropout(rate = 0.3)(dec_l4)

    decoder_output = Dense(input_shape, activation='sigmoid', name='decoder_output')(dec_l4)

    model = Model(inputs=[enc_input], outputs=[encoder_output, decoder_output])

    # Compiling model
    model.compile(optimizer='adam',#optimizer='rmsprop',
                  loss={'encoder_output': 'categorical_crossentropy', 'decoder_output': 'mean_squared_error'},
                  loss_weights={'encoder_output': 0.5, 'decoder_output': 0.5},
                  metrics=[metrics.categorical_accuracy])
    return model

In [ ]:
## Designing an Auto-Encoder-Classifier model
def Deep_SRC_Decoder(input_shape =600, num_features=2400):
    # Encoder Network
    dec_input = Input(shape=(input_shape,), name='decoder_input')

    # Decoder Network
    dec_l1 = Dense(100, activation='relu', name='decoder_layer1')(dec_input)
    dec_l1 = BatchNormalization()(dec_l1)
    dec_l1 = Dropout(rate = 0.3)(dec_l1)

    dec_l2 = Dense(100, activation='relu', name='decoder_layer2')(dec_l1)
    dec_l2 = BatchNormalization()(dec_l2)
    dec_l2 = Dropout(rate = 0.3)(dec_l2)

    dec_l3 = Dense(100, activation='relu', name='decoder_layer3')(dec_l2)
    dec_l3 = BatchNormalization()(dec_l3)
    dec_l3 = Dropout(rate = 0.3)(dec_l3)

    dec_l4 = Dense(100, activation='relu', name='decoder_layer4')(dec_l3)
    dec_l4 = BatchNormalization()(dec_l4)
    dec_l4 = Dropout(rate = 0.3)(dec_l4)

    decoder_output = Dense(num_features, activation='sigmoid', name='decoder_output')(dec_l4)

    model = Model(inputs=[dec_input], outputs=[decoder_output])

    # Compiling model
    model.compile(optimizer='adam',#optimizer='rmsprop',
                  loss={'decoder_output': 'mean_squared_error'},
                  metrics=[metrics.mean_squared_error])
    return model

In [ ]:
## Define CKSAAP feature-extraction function
def minSequenceLength(fastas):
	minLen = 10000
	for i in fastas:
		if minLen > len(i[1]):
			minLen = len(i[1])
	return minLen

def CKSAAP(fastas, gap=5, **kw):
	if gap < 0:
		print('Error: the gap should be equal or greater than zero' + '\n\n')
		return 0

	if minSequenceLength(fastas) < gap+2:
		print('Error: all the sequence length should be larger than the (gap value) + 2 = ' + str(gap+2) + '\n\n')
		return 0

	AA = 'ACDEFGHIKLMNPQRSTVWY'
	encodings = []
	aaPairs = []
	for aa1 in AA:
		for aa2 in AA:
			aaPairs.append(aa1 + aa2)
	header = ['#']
	for g in range(gap+1):
		for aa in aaPairs:
			header.append(aa + '.gap' + str(g))
	encodings.append(header)
	for i in fastas:
		name, sequence = i[0], i[1]
		code = [name]
		for g in range(gap+1):
			myDict = {}
			for pair in aaPairs:
				myDict[pair] = 0
			sum = 0
			for index1 in range(len(sequence)):
				index2 = index1 + g + 1
				if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[index2] in AA:
					myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
					sum = sum + 1
			for pair in aaPairs:
				code.append(myDict[pair] / sum)
		encodings.append(code)
	return encodings

In [ ]:
def load_seq_data(data_path,label):
  dataset = pd.read_csv(data_path,names=None,index_col=0, header=None)
  seq = []
  sample_count = 0

  for row in dataset.iterrows():
    if(row[0]!='>'):
      sample_count = sample_count +1
      array = [label, row[0]]
      name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
      seq.append([name, sequence])

  print('# of ' + label + ' samples',sample_count)
  return seq

In [ ]:
def prepare_feature_acp240_740(path = r"acp240.txt"):
    # path = r"acp740.txt"
    # path = r"acp240.txt"
    new_list=[]
    seq_list=[]
    label = []
    lis = []
    interaction_pair = {}
    RNA_seq_dict = {}
    protein_seq_dict = {}
    protein_index = 0
    with open(path, 'r') as fp:
        for line in fp:
            if line[0] == '>':
                values = line[1:].strip().split('|')
                label_temp = values[1]
                proteinName = values[0]
                proteinName_1=proteinName.split("_")
                new_list.append(proteinName_1[0])
    #             print(new_list)

                if label_temp == '1':
                    label.append(1)
                else:
                    label.append(0)
            else:
                seq = line[:-1]
                seq_list.append(seq)
        for i, item in enumerate(new_list):
            lis.append([item, seq_list[i]])

    return lis


In [ ]:
def Convert_Seq2CKSAAP(train_seq, gap=8):
  cksaapfea = []
  seq_label = []
  for sseq in train_seq:
    temp= CKSAAP([sseq], gap=gap)
    cksaapfea.append(temp[1][1:])
    seq_label.append(sseq[0])

  x = np.array(cksaapfea)
  y = np.array(seq_label)
  y[y=='ACP']=0
  y[y=='non-ACP']=1
  y[y=='POS']=0
  y[y=='NEG']=1
  y = to_categorical(y)
  print('num pos:', sum(y[:,0]==1), 'num neg:', sum(y[:,0]==0))
  return x,y

def minSequenceLength(fastas):
    minLen = 10000
    for i in fastas:
        if minLen > len(i[1]):
            minLen = len(i[1])
    return minLen

def CKSAAP(fastas, gap=5, **kw):
    if gap < 0:
        print('Error: the gap should be equal or greater than zero' + '\n\n')
        return 0

    if minSequenceLength(fastas) < gap+2:
        print('Error: all the sequence length should be larger than the (gap value) + 2 = ' + str(gap+2) + '\n' + 'Current sequence length ='  + str(minSequenceLength(fastas)) + '\n\n')
        return 0

    AA = 'ACDEFGHIKLMNPQRSTVWY'
    encodings = []
    aaPairs = []
    for aa1 in AA:
        for aa2 in AA:
            aaPairs.append(aa1 + aa2)
    header = ['#']
    for g in range(gap+1):
        for aa in aaPairs:
            header.append(aa + '.gap' + str(g))
    encodings.append(header)
    for i in fastas:
        name, sequence = i[0], i[1]
        code = [name]
        for g in range(gap+1):
            myDict = {}
            for pair in aaPairs:
                myDict[pair] = 0
            sum = 0
            for index1 in range(len(sequence)):
                index2 = index1 + g + 1
                if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[index2] in AA:
                    myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
                    sum = sum + 1
            for pair in aaPairs:
                code.append(myDict[pair] / sum)
        encodings.append(code)
    return encodings

In [ ]:
# ## Loading and pre-processing of dataset
pos_all_seq_path = 'https://raw.githubusercontent.com/Shujaat123/ACP_LSE/main/dataset_acp_JTB_2014/1-s2.0-S0022519313004190-mmc1.txt'
neg_all_seq_path = 'https://raw.githubusercontent.com/Shujaat123/ACP_LSE/main/dataset_acp_JTB_2014/1-s2.0-S0022519313004190-mmc2.txt'

!pip install wget
import wget
dataset_path='https://raw.githubusercontent.com/haichengyi/ACP-DL/master/acp740.txt'
wget.download(dataset_path, 'acp740.txt')

dataset_path='https://raw.githubusercontent.com/haichengyi/ACP-DL/master/acp240.txt'
wget.download(dataset_path, 'acp240.txt')

!pip install mrmr_selection
from mrmr import mrmr_classif

pos_all_seq = load_seq_data(pos_all_seq_path,'POS')
neg_all_seq = load_seq_data(neg_all_seq_path,'NEG')

ALL_seq344 = pos_all_seq + neg_all_seq

print(len(pos_all_seq), len(neg_all_seq), len(ALL_seq344))

ALL_seq240=prepare_feature_acp240_740(path = r"acp240.txt")
ALL_seq740=prepare_feature_acp240_740(path = r"acp740.txt")

print(len(ALL_seq240))
print(len(ALL_seq344))
print(len(ALL_seq740))

# of POS samples 138
# of NEG samples 206
138 206 344
240
344
740


In [ ]:
# train_set = pd.read_csv("https://raw.githubusercontent.com/Shujaat123/AFP-SRC/master/data/train1.csv")
# test_set = pd.read_csv("https://raw.githubusercontent.com/Shujaat123/AFP-SRC/master/data/test1.csv")

In [ ]:
# # from keras.utils.np_utils import to_categorical
# X_train = train_set.iloc[:, 1:].to_numpy()
# y_train = np.asarray(train_set.CLASS)
# y_train[y_train=='AFP']=1
# y_train[y_train=='NON_AFP']=0
# y_train = to_categorical(y_train)

# X_test = test_set.iloc[:, 1:].to_numpy()
# y_test = np.asarray(test_set.CLASS)
# y_test[y_test=='AFP']=1
# y_test[y_test=='NON_AFP']=0
# y_test = to_categorical(y_test)

# print('X_train.shape:',X_train.shape,'X_test.shape:',X_test.shape)
# print('y_train.shape:',y_train.shape,'y_test.shape:',y_test.shape)




In [ ]:
def sample_one_hot_encoder(label):
  ntrain = len(y_train)
  onehot_encoded = list()
  for value in range(ntrain):
    letter = [0 for _ in range(ntrain)]
    letter[value] = 1
    onehot_encoded.append(letter)

  return np.array(onehot_encoded)

In [ ]:
# y_train_latent = sample_one_hot_encoder(y_train)
# y_train_latent.shape[1]

In [ ]:
import numpy.linalg as LA
def SRC_Pred(y_latent_pred):
    y_pred=[]
    for i in range(y_latent_pred.shape[0]):

        if (LA.norm(y_latent_pred[i][y_train.argmax(axis=1)==1])>= LA.norm(y_latent_pred[i][y_train.argmax(axis=1)==0])):
            y_pred.append(1)
        else:
            y_pred.append(0)

    return to_categorical(np.array(y_pred))

In [ ]:
# y_train_latent_pred.shape, y_test_latent_pred.shape
# y_train_pred = SRC_Pred_Dict_Recon(y_train, y_train_latent_pred, X_train, X_train)
# y_test_pred = SRC_Pred_Dict_Recon(y_test, y_test_latent_pred, X_train, X_test)

In [ ]:
def custom_softmax(x):
  exp_x = np.exp(x - np.max(x))
  return exp_x / np.sum(exp_x)
x = np.array([1, 2, 4, 5])
custom_softmax(x)

array([0.01275478, 0.03467109, 0.25618664, 0.69638749])

In [ ]:
def SRC_Pred_Dict_Recon(y_train, y_latent_pred, X_train, X_test):
  npos = (y_train.argmax(axis=1)==0).sum()
  nneg = len(y_train) - npos
  y_latent_pred_pos = []
  y_latent_pred_neg = []
  for i in range(y_latent_pred.shape[0]):
     y_latent_pred_pos.append(np.concatenate((np.array(custom_softmax(y_latent_pred[i]))[y_train.argmax(axis=1)==0], np.repeat(0,nneg)), axis=0))
     y_latent_pred_neg.append(np.concatenate((np.repeat(0,npos), np.array(custom_softmax(y_latent_pred[i]))[y_train.argmax(axis=1)==1]), axis=0))

  X_pred_pos = np.matmul(np.array(y_latent_pred_pos), X_train)
  X_pred_neg = np.matmul(np.array(y_latent_pred_neg), X_train)

  y_pred=[]
  for i in range(X_test.shape[0]):
    pos_err = np.square(X_test[i] - X_pred_pos).sum()
    neg_err = np.square(X_test[i] - X_pred_neg).sum()
    if pos_err <= neg_err:
      y_pred.append(0)
    else:
      y_pred.append(1)

  return to_categorical(np.array(y_pred))

In [ ]:
# y_latent_pred = np.random.rand(len(y_train),len(y_train))
# y_pred = SRC_Pred(y_latent_pred)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [ ]:
def SRC_Pred_Dec_Recon(y_train, y_latent_pred, model, X_test, argmax=False):
  npos = (y_train.argmax(axis=1)==0).sum()
  nneg = len(y_train) - npos
  y_latent_pred_pos = []
  y_latent_pred_neg = []
  for i in range(y_latent_pred.shape[0]):
    if argmax:
      tmp_y_latent_pred_pos = np.repeat(0, len(y_latent_pred[i]))
      tmp_y_latent_pred_pos[y_latent_pred[i][y_train==0].argmax(axis=1)] = 1
      tmp_y_latent_pred_neg = np.repeat(0, len(y_latent_pred[i]))
      tmp_y_latent_pred_neg[y_latent_pred[i][y_train==1].argmax(axis=1)] = 1
      y_latent_pred_pos.append(tmp_y_latent_pred_pos)
      y_latent_pred_neg.append(tmp_y_latent_pred_neg)
    else:
     y_latent_pred_pos.append(np.concatenate((np.array(custom_softmax(y_latent_pred[i]))[y_train.argmax(axis=1)==0], np.repeat(0,nneg)), axis=0))
     y_latent_pred_neg.append(np.concatenate((np.repeat(0,npos), np.array(custom_softmax(y_latent_pred[i]))[y_train.argmax(axis=1)==1]), axis=0))

  y_latent_pred_pos = np.array(y_latent_pred_pos)
  y_latent_pred_neg = np.array(y_latent_pred_neg)
  X_pred_pos = model_dec.predict(y_latent_pred_pos)
  X_pred_neg = model_dec.predict(y_latent_pred_neg)

  # print('y_latent_pred_neg.shape:',y_latent_pred_neg.shape)

  y_pred=[]
  # print('X_test.shape[0]:',X_test.shape[0])
  for i in range(X_test.shape[0]):
    # print('X_test.shape:',X_test.shape)
    # print('X_pred_pos.shape:',X_pred_pos.shape)
    # print('X_pred_neg.shape:',X_pred_neg.shape)
    pos_err = np.square(X_test[i] - X_pred_pos[i]).sum()
    neg_err = np.square(X_test[i] - X_pred_neg[i]).sum()
    if pos_err <= neg_err:
      y_pred.append(0)
    else:
      y_pred.append(1)
  return np.array(y_pred)
  # return to_categorical(np.array(y_pred))

# print('X_train.shape:',X_train.shape)
# print(10*'*')
# y_train_src_rec_pred = SRC_Pred_Dec_Recon(y_train, y_train_latent_pred, model_dec, X_train)

In [ ]:
def yoden_index(y, y_pred):
  epsilon = 1e-30
  tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
  j = (tp/(tp + fn + epsilon)) + (tn/(tn+fp + epsilon)) - 1
  return j

def pmeasure(y, y_pred):
    epsilon = 1e-30
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn + epsilon)
    specificity = tn / (tn + fp + epsilon)
    f1score = (2 * tp) / (2 * tp + fp + fn + epsilon)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

In [ ]:
def Calculate_Stats(y_actual,y_pred):
  acc = accuracy_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  sen = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['Sensitivity']
  spe = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['Specificity']
  f1 = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['F1-Score']
  mcc = matthews_corrcoef(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  bacc = balanced_accuracy_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  yi = yoden_index(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  #auc = roc_auc_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))

  #pre, rec, _ = precision_recall_curve(y_actual.argmax(axis=1), y_score, pos_label=1)
  #fpr, tpr, _ = roc_curve(y_actual.argmax(axis=1), y_score, pos_label=1)
  #auroc = auc(fpr, tpr)
  #aupr = auc(rec, pre)

  return acc, sen, spe, f1, mcc, bacc, yi

In [ ]:
def add_dimensions(pred_array):
  if(len(pred_array.shape)==1):
    pred_array = np.expand_dims(pred_array,axis=1)
    pred_array = np.concatenate((pred_array,np.abs(1-pred_array)),axis=1)
  return pred_array

In [ ]:
def Show_Statistics(msg,mean_Stats, sd_Stats, sigfig):
  print(msg.upper())
  print(70*'-')
  print('Accuracy:{} + {}'          .format(round(mean_Stats[0],sigfig), round(sd_Stats[0],sigfig)))
  print('Sensitivity:{} + {} '      .format(round(mean_Stats[1],sigfig), round(sd_Stats[1],sigfig)))
  print('Specificity:{} + {}'       .format(round(mean_Stats[2],sigfig), round(sd_Stats[2],sigfig)))
  print('F1-Score:{} + {}'          .format(round(mean_Stats[3],sigfig), round(sd_Stats[3],sigfig)))
  print('MCC:{} + {}'               .format(round(mean_Stats[4],sigfig), round(sd_Stats[4],sigfig)))
  print('Balance Accuracy:{} + {}'  .format(round(mean_Stats[5],sigfig), round(sd_Stats[5],sigfig)))
  print('Youden-Index:{} + {}'      .format(round(mean_Stats[6],sigfig), round(sd_Stats[6],sigfig)))
  print(70*'-')

In [ ]:
[DataX, LabelY] = Convert_Seq2CKSAAP(ALL_seq344, gap=5)
from sklearn.model_selection import KFold, StratifiedKFold
kf = StratifiedKFold(n_splits=2,shuffle=True, random_state=25)
loop_ind = 0
for train_index, test_index in kf.split(DataX,LabelY.argmax(axis=1)):
  loop_ind = loop_ind + 1
  X_train, X_test = DataX[train_index], DataX[test_index]
  y_train, y_test = LabelY[train_index], LabelY[test_index]
  print('X_train.shape:',X_train.shape,'X_test.shape:',X_test.shape)
  print('y_train.shape:',y_train.shape,'y_test.shape:',y_test.shape)

  #############################################
  y_train_latent = sample_one_hot_encoder(y_train)
  y_train_latent.shape[1]
  #############################################

  model = Deep_SRC(input_shape = X_train.shape[1], LV=len(y_train))
  #model = Deep_VAE_SRC(input_shape = X_train.shape[1], LV=len(y_train))
  es = EarlyStopping(monitor='encoder_output_categorical_accuracy', mode='max', verbose=0, patience=100)
  checkpoint = ModelCheckpoint('models\\model-best.h5',
                              verbose=0, monitor='encoder_output_categorical_accuracy',save_best_only=True, mode='auto')

  history = model.fit({'enc_input': X_train},
                      {'encoder_output': y_train_latent, 'decoder_output': X_train},
                      # validation_data = ({'enc_input': X_val},
                      # {'class_output': y_val, 'decoder_output': X_val}),
                      epochs=5000, batch_size=420, callbacks=[checkpoint, es], verbose=0)

  y_train_latent_pred, X_train_pred = model.predict(X_train,batch_size=540, verbose=0)
  #y_val_latent_pred, X_val_pred = model.predict(X_val,batch_size=540, verbose=0)
  y_test_latent_pred, X_test_pred = model.predict(X_test,batch_size=540, verbose=0)

  print('y_test_latent_pred.shape:',y_test_latent_pred.shape)
  #############################################################
  #############################################################
  #del model
  #model = load_model('models\\model-best.h5')

  model_dec = Deep_SRC_Decoder(input_shape = len(y_train), num_features=X_train.shape[1])

  # es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=100)
  # checkpoint = ModelCheckpoint('models\\model_dec-best.h5',
  #                              verbose=0, monitor='loss',save_best_only=True, mode='auto')

  # history = model_dec.fit({'decoder_input': y_train_latent},
  #                     {'decoder_output': X_train},
  #                     # validation_data = ({'enc_input': X_val},
  #                     # {'class_output': y_val, 'decoder_output': X_val}),
  #                     epochs=5000, batch_size=600, callbacks=[checkpoint, es], verbose=0)
  # del model_dec
  # model_dec = load_model('models\\model_dec-best.h5')

  for i in range(1,12):
      model_dec.layers[i].set_weights(model.layers[13+i].get_weights())

  ## By Norm
  y_train_src_pred = SRC_Pred(y_train_latent_pred)
  y_test_src_pred = SRC_Pred(y_test_latent_pred)
  train_src_pred_acc = accuracy_score(y_train, y_train_src_pred)
  print('y_test.shape:',y_test.shape,'y_test_src_pred.shape',y_test_src_pred.shape)
  test_src_pred_acc = accuracy_score(y_test, y_test_src_pred)

  ## By Dictionary (Train Dataset)
  y_train_src_dict_pred = SRC_Pred_Dict_Recon(y_train, y_train_latent_pred, X_train, X_train)
  y_test_src_dict_pred = SRC_Pred_Dict_Recon(y_train, y_test_latent_pred, X_train, X_test)
  train_src_dict_acc = accuracy_score(y_train, y_train_src_dict_pred)
  test_src_dict_acc = accuracy_score(y_test, y_test_src_dict_pred)
  ################################################################
  ################################################################
  ## By Reconstruction (Model)
  print("By Reconstruction (Model)")
  y_train_src_rec_pred = SRC_Pred_Dec_Recon(y_train, y_train_latent_pred, model_dec, X_train)
  y_test_src_rec_pred = SRC_Pred_Dec_Recon(y_train, y_test_latent_pred, model_dec, X_test)

  y_train_src_rec_pred = add_dimensions(y_train_src_rec_pred)
  y_test_src_rec_pred = add_dimensions(y_test_src_rec_pred)

  #y_train_src_rec_pred = SRC_Pred_Dec_Recon(y_train, y_train_latent_pred, model_dec, X_train, argmax=True)
  #y_test_src_rec_pred = SRC_Pred_Dec_Recon(y_train, y_test_latent_pred, model_dec, X_test, argmax=True)
  print('y_train.shape:',y_train.shape,'y_train_src_rec_pred.shape',y_train_src_rec_pred.shape)
  train_src_rec_acc = accuracy_score(y_train,y_train_src_rec_pred)
  # train_src_rec_acc = accuracy_score(y_train.argmax(axis=1),y_train_src_rec_pred)
  test_src_rec_acc = accuracy_score(y_test,y_test_src_rec_pred)
  # test_src_rec_acc = accuracy_score(y_test.argmax(axis=1),y_test_src_rec_pred)

  print("train_acc_by_Norm: {}, test_acc_by_Norm: {}".format(train_src_pred_acc, test_src_pred_acc))
  print("train_acc_by_Dict: {}, test_acc_by_Dict: {}".format(train_src_dict_acc, test_src_dict_acc))
  print("train_acc_by_Recon: {}, test_acc_by_Recon: {}".format(train_src_rec_acc, test_src_rec_acc))

  ## Input Train loss
  MSE_X_train_pred = -10*np.log10(np.mean(np.square(X_train_pred - X_train)))
  #MSE_X_val_pred = -10*np.log10(np.mean(np.square(X_val_pred - X_val)))
  MSE_X_test_pred = -10*np.log10(np.mean(np.square(X_test_pred - X_test)))

  ## Sample Label loss
  MSE_y_latent_train_pred = -10*np.log10(np.mean(np.square(y_train_latent_pred - y_train_latent)))

  print(MSE_X_train_pred, MSE_y_latent_train_pred)

  ################################################################################
  ################################################################################
  train_list = [y_train_src_pred, y_train_src_dict_pred, y_train_src_rec_pred]
  test_list = [y_test_src_pred, y_test_src_dict_pred, y_test_src_rec_pred]
  ################################################################################
  ################################################################################
  Stats=[]

  for i in range(3):
    y_train_pred = train_list[i]
    y_test_pred = test_list[i]

    ## Training Measures
    tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi = Calculate_Stats(y_train, y_train_pred);

    ## Validation Measures
    #v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi = Calculate_Stats(to_categorical(y_val),y_val_pred);

    ## Test Measures
    t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi = Calculate_Stats(y_test,y_test_pred);

    Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi,
                  #              v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi,
                  t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi])

  Statistics = np.asarray(Stats)

  ################################################################################
  ################################################################################
  Show_Statistics('Norm Training Results (MEAN)',Statistics[0][0:7],Statistics.std(axis=0)[0:7], 3)
  Show_Statistics('Norm Test Results (MEAN)',Statistics[0][7:14],Statistics.std(axis=0)[7:14], 3)
  Show_Statistics('Dict Training Results (MEAN)',Statistics[1][0:7],Statistics.std(axis=0)[0:7], 3)
  Show_Statistics('Dict Test Results (MEAN)',Statistics[1][7:14],Statistics.std(axis=0)[7:14], 3)
  Show_Statistics('Rec Training Results (MEAN)',Statistics[2][0:7],Statistics.std(axis=0)[0:7], 3)
  Show_Statistics('Rec Test Results (MEAN)',Statistics[2][7:14],Statistics.std(axis=0)[7:14], 3)
  #Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[14:21],Statistics.std(axis=0)[14:21], 3)

num pos: 138 num neg: 206
X_train.shape: (172, 2400) X_test.shape: (172, 2400)
y_train.shape: (172, 2) y_test.shape: (172, 2)
y_test_latent_pred.shape: (172, 172)
y_test.shape: (172, 2) y_test_src_pred.shape (172, 2)
By Reconstruction (Model)
6/6 [==============================] - 0s 3ms/step
y_train.shape: (172, 2) y_train_src_rec_pred.shape (172, 2)
train_acc_by_Norm: 0.5988372093023255, test_acc_by_Norm: 0.5988372093023255
train_acc_by_Dict: 0.9302325581395349, test_acc_by_Dict: 0.9069767441860465
train_acc_by_Recon: 0.5988372093023255, test_acc_by_Recon: 0.5988372093023255
21.622006001870805 22.377929749618986
NORM TRAINING RESULTS (MEAN)
----------------------------------------------------------------------
Accuracy:0.599 + 0.156
Sensitivity:1.0 + 0.009 
Specificity:0.0 + 0.403
F1-Score:0.749 + 0.092
MCC:0.0 + 0.404
Balance Accuracy:0.5 + 0.197
Youden-Index:0.0 + 0.394
----------------------------------------------------------------------
NORM TEST RESULTS (MEAN)
-----------------

Before this it is working

In [ ]:
# train_list = [y_train_src_pred, y_train_src_dict_pred, y_train_src_rec_pred]
# test_list = [y_test_src_pred, y_test_src_dict_pred, y_test_src_rec_pred]

In [ ]:
# ################################################################################
# ################################################################################
# train_list = [y_train_src_pred, y_train_src_dict_pred, y_train_src_rec_pred]
# test_list = [y_test_src_pred, y_test_src_dict_pred, y_test_src_rec_pred]
# ################################################################################
# ################################################################################
# Stats=[]

# for i in range(3):
#   y_train_pred = train_list[i]
#   y_test_pred = test_list[i]

#   ## Training Measures
#   tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi = Calculate_Stats(y_train, y_train_pred);

#   ## Validation Measures
#   #v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi = Calculate_Stats(to_categorical(y_val),y_val_pred);

#   ## Test Measures
#   t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi = Calculate_Stats(y_test,y_test_pred);

#   Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi,
#                 #              v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi,
#                 t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi])

# Statistics = np.asarray(Stats)

# ################################################################################
# ################################################################################
# Show_Statistics('Norm Training Results (MEAN)',Statistics[0][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Norm Test Results (MEAN)',Statistics[0][7:14],Statistics.std(axis=0)[7:14], 3)
# Show_Statistics('Dict Training Results (MEAN)',Statistics[1][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Dict Test Results (MEAN)',Statistics[1][7:14],Statistics.std(axis=0)[7:14], 3)
# Show_Statistics('Rec Training Results (MEAN)',Statistics[2][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Rec Test Results (MEAN)',Statistics[2][7:14],Statistics.std(axis=0)[7:14], 3)
# #Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[14:21],Statistics.std(axis=0)[14:21], 3)

In [ ]:
# Show_Statistics('Norm Training Results (MEAN)',Statistics[0][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Norm Test Results (MEAN)',Statistics[0][7:14],Statistics.std(axis=0)[7:14], 3)
# Show_Statistics('Dict Training Results (MEAN)',Statistics[1][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Dict Test Results (MEAN)',Statistics[1][7:14],Statistics.std(axis=0)[7:14], 3)
# Show_Statistics('Rec Training Results (MEAN)',Statistics[2][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Rec Test Results (MEAN)',Statistics[2][7:14],Statistics.std(axis=0)[7:14], 3)
# #Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[14:21],Statistics.std(axis=0)[14:21], 3)

NORM TRAINING RESULTS (MEAN)
----------------------------------------------------------------------
Accuracy:0.599 + 0.234
Sensitivity:1.0 + 0.469 
Specificity:0.0 + 0.455
F1-Score:0.749 + 0.416
MCC:0.0 + 0.437
Balance Accuracy:0.5 + 0.216
Youden-Index:0.0 + 0.433
----------------------------------------------------------------------
NORM TEST RESULTS (MEAN)
----------------------------------------------------------------------
Accuracy:0.599 + 0.186
Sensitivity:1.0 + 0.463 
Specificity:0.0 + 0.419
F1-Score:0.749 + 0.39
MCC:0.0 + 0.33
Balance Accuracy:0.5 + 0.155
Youden-Index:0.0 + 0.31
----------------------------------------------------------------------
DICT TRAINING RESULTS (MEAN)
----------------------------------------------------------------------
Accuracy:0.965 + 0.234
Sensitivity:0.99 + 0.469 
Specificity:0.928 + 0.455
F1-Score:0.971 + 0.416
MCC:0.928 + 0.437
Balance Accuracy:0.959 + 0.216
Youden-Index:0.918 + 0.433
-------------------------------------------------------------